In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import json
# , num_labels=4, ignore_mismatched_sizes
model = AutoModelForSequenceClassification.from_pretrained('cross-encoder/ms-marco-MiniLM-L-12-v2')
tokenizer = AutoTokenizer.from_pretrained('cross-encoder/ms-marco-MiniLM-L-12-v2')

In [68]:
s = open("../prompt_engineering/gpt-3_semantic_search/1_top_quality.json")
first = json.load(s)
s = open("../prompt_engineering/gpt-3_semantic_search/2_decent_enough_to_keep.json")
second = json.load(s)
s = open("../prompt_engineering/gpt-3_semantic_search/3_to_delete.json")
third = json.load(s)
s = open("../prompt_engineering/gpt-3_semantic_search/4_invalid_items.json")
fourth = json.load(s)

In [113]:
questions = []
answers = []
labels = []
qa = []
for n, dataset in enumerate([first, second, third, fourth]):
    for row in dataset:
        labels.append(n)
        questions.append(row['GPT-3-Semantic-Search-Generations']['question'])
        answers.append(row['GPT-3-Semantic-Search-Generations']['answer'])
        qa.append(row['GPT-3-Semantic-Search-Generations']['question'] + row['GPT-3-Semantic-Search-Generations']['answer'])

In [114]:
from sklearn.model_selection import train_test_split
train_qa, test_qa, train_labels, test_labels = train_test_split(qa, labels, test_size=.2)

In [115]:
encoded_train_qa = tokenizer(train_qa, truncation=True, padding=True)
encoded_test_qa = tokenizer(test_qa, truncation=True, padding=True)

In [116]:
import torch

class filtered_data(torch.utils.data.Dataset):
    def __init__(, encodings, labels):
        .encodings = encodings
        .labels = labels

    def __getitem__(, idx):
        item = {key: torch.tensor(val[idx]) for key, val in .encodings.items()}
        item['labels'] = torch.tensor(.labels[idx])
        return item

    def __len__():
        return len(.labels)

In [117]:
train_data = filtered_data(encoded_train_qa, train_labels)
test_data = filtered_data(encoded_test_qa, test_labels)

In [118]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [119]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [120]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    tokenizer=tokenizer
)

In [121]:
trainer.train()

***** Running training *****
  Num examples = 133
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 51


  0%|          | 0/51 [00:00<?, ?it/s]

***** Running Evaluation *****
  Num examples = 34
  Batch size = 8


  0%|          | 0/5 [00:00<?, ?it/s]

{'eval_loss': 1.2938010692596436, 'eval_runtime': 1.092, 'eval_samples_per_second': 31.136, 'eval_steps_per_second': 4.579, 'epoch': 1.0}


...

***** Running Evaluation *****
  Num examples = 34
  Batch size = 8


  0%|          | 0/5 [00:00<?, ?it/s]



Training completed. Do not forget to share your model on huggingface.co/models =)




{'eval_loss': 1.1651407480239868, 'eval_runtime': 1.1235, 'eval_samples_per_second': 30.262, 'eval_steps_per_second': 4.45, 'epoch': 3.0}
{'train_runtime': 45.1822, 'train_samples_per_second': 8.831, 'train_steps_per_second': 1.129, 'train_loss': 1.2540463466270297, 'epoch': 3.0}


TrainOutput(global_step=51, training_loss=1.2540463466270297, metrics={'train_runtime': 45.1822, 'train_samples_per_second': 8.831, 'train_steps_per_second': 1.129, 'train_loss': 1.2540463466270297, 'epoch': 3.0})

In [125]:
import numpy as np
# import evaluate

metric = evaluate.load("accuracy")

In [126]:
print(metric)

EvaluationModule(name: "accuracy", module_type: "metric", features: {'predictions': Value(dtype='int32', id=None), 'references': Value(dtype='int32', id=None)}, usage: """
Args:
    predictions (`list` of `int`): Predicted labels.
    references (`list` of `int`): Ground truth labels.
    normalize (`boolean`): If set to False, returns the number of correctly classified samples. Otherwise, returns the fraction of correctly classified samples. Defaults to True.
    sample_weight (`list` of `float`): Sample weights Defaults to None.

Returns:
    accuracy (`float` or `int`): Accuracy score. Minimum possible value is 0. Maximum possible value is 1.0, or the number of examples input, if `normalize` is set to `True`.. A higher score means higher accuracy.

Examples:

    Example 1-A simple example
        >>> accuracy_metric = evaluate.load("accuracy")
        >>> results = accuracy_metric.compute(references=[0, 1, 2, 0, 1, 2], predictions=[0, 1, 1, 2, 1, 0])
        >>> print(results)
    

In [2]:
import pytorch_lightning as pl

In [2]:
import json
import random
s = open("/Users/joshuamin/Desktop/Internships/UIUC_chatbot_data_generator/prompt_engineering/gpt-3_semantic_search/1_top_quality.json")
first = json.load(s)
s = open("/Users/joshuamin/Desktop/Internships/UIUC_chatbot_data_generator//prompt_engineering/gpt-3_semantic_search/2_decent_enough_to_keep.json")
second = json.load(s)
s = open("/Users/joshuamin/Desktop/Internships/UIUC_chatbot_data_generator//prompt_engineering/gpt-3_semantic_search/3_to_delete.json")
third = json.load(s)
s = open("/Users/joshuamin/Desktop/Internships/UIUC_chatbot_data_generator//prompt_engineering/gpt-3_semantic_search/4_invalid_items.json")
fourth = json.load(s)

In [3]:
bad_data = []
for dataset in [third, fourth]:
    for row in dataset:
        bad_data.append(row['GPT-3-Semantic-Search-Generations']['answer'])

triplets = []
for dataset in [first, second]:
    for row in dataset:
        "TODO equality check for negs"
        triplets.append([row['GPT-3-Semantic-Search-Generations']['question'], row['GPT-3-Semantic-Search-Generations']['answer'],[random.choice(bad_data), random.choice(bad_data), random.choice(bad_data)]])


In [4]:
query_doc_pairs = [[] for _ in  range(4)]
import pprint

for row in triplets:
    # TODO @josh
    # row[0] = query
    # row[1] = pos
    # row[2] = neg
    query_text = row[0]
    # pos
    query_doc_pairs[0].append((query_text, row[1]))
    # negs
    for neg_id, neg in enumerate(row[2]):
        query_doc_pairs[1+neg_id].append((query_text, neg))
for row in query_doc_pairs:
    pprint.pprint(row)
    print('\n\n\n\n\n')

[('What is the design process for a digital FSM?\\n',
  '\n'
  'The design process for a digital FSM includes six steps: developing an '
  'abstract model, specifying I/O behavior, completing the specification, '
  'choosing a state representation, calculating logic expressions, and '
  'implementing with flip-flops and gates.'),
 ('What is the need for FSM initialization?\\n',
  '\n'
  'An FSM must be initialized to ensure that it starts in a known state and to '
  'avoid undefined behavior. Without initialization, an FSM may produce '
  'incorrect results or may not work at all.'),
 ('What is a multiplexer?\\n',
  '\n'
  'A multiplexer is a digital logic block that allows one digital signal to '
  'select which of several others is forwarded to an output. The simplest form '
  'of the multiplexer is the 2-to-1 multiplexer, which has two inputs from the '
  'left and one from the top. The top input allows us to choose which of the '
  'left inputs is forwarded to the output.'),
 ('Wha